In [16]:
from sympy import symbols, Matrix
from sympy import expand, factor #expand and factor expressions
import random
import numpy as np
from sympy import Matrix,eye,det,zeros

[Sympy doc] (https://docs.sympy.org/latest/tutorials/intro-tutorial/index.html)

1. Symbolic computation: manipulating irrational numbers exactly using SymPy, computing symbolic expressions with variables.
2. most simplifications in Sympy are not performed automatically.
3. SymPy can simplify expressions, compute derivatives, integrals, and limits, solve equations, work with matrices, and much, much more, and do it all symbolically.



In [4]:
x,y=symbols('x y')
expr=x+2*y
expanded_expr=expand(x*expr)
factor(expanded_expr)

x*(x + 2*y)

some rules:
* To define variables, we must use symbols.
* if a variable is changed, expressions that were already created with that variable do not change automatically
* == represents exact structural equality testing. “Exact” here means that two expressions will compare equal with == only if they are exactly equal structurally.
* the best way to check whether a-b: subtract, simplify(b-a), and see whether you get 0 or a.equals(b)

In [33]:
x = symbols('x')
expr = x + 1
# x = 2
#print(expr)#changes made to x has no effects in expr, which is created before the chg
expr.subs(x, 2)

3

Converting Strings to SymPy Expressions
**sympify**, not to be confused with simplify

# Key Generation

In [2]:
def poly_n_variables(n):
    K=[]
    for i in range(n):
        x='x'+str(i)
        K.append(x)
    return K

def sparse_polynomial(n,t,b,q):#t monomials, bound the degree of monomials by an integer b
    K=poly_n_variables(n)
    sparse_poly=0
    
    for i in range(t):
        deg=random.randint(1,b)
        monomoial_d=1
        
        for i in range(deg):
            monomoial_d*=symbols(random.choice(K))
        sparse_poly+=random.randint(1,q-1)*monomoial_d
    return sparse_poly

sparse_polynomial(32,5,5,6)

2*x0 + x14*x17*x9**2 + 5*x14*x29*x7 + 2*x16 + x18*x22

# Generate the matrix

In [6]:
def generate_U(k,n,t,B,q):
    I=eye(k)
    
    for i in range(int((k*k-k)/2)):
        b=random.randint(1,k-1)
        a=random.randint(0,b-1)
        I[a,b]=sparse_polynomial(n,t,B,q)
    return I
    
def generate_L(k,n,t,B,q):
    I=eye(k)
    
    for i in range(int((k*k-k)/2)):
        a=random.randint(1,k-1)
        b=random.randint(0,a-1)
        I[a,b]=sparse_polynomial(n,t,B,q)
    return I

A=generate_U(4,32,5,6,6)
B=A.inv(method="LU")
A*B

Matrix([
[1, 0, 0, 0],
[0, 1, 0, 0],
[0, 0, 1, 0],
[0, 0, 0, 1]])

In [17]:
zeros(3)

Matrix([
[0, 0, 0],
[0, 0, 0],
[0, 0, 0]])

In [28]:

def col_permutation(matrix):
    col_permutation = random.sample(range(matrix.cols), matrix.cols)
#     permuted_matrix = matrix.permute_cols(col_permutation)
    
    I=zeros(len(col_permutation))
    for i in range(len(col_permutation)):
        I[i,col_permutation[i]]=1
#     print(I.inv())
#     print(matrix*I.inv())
    
    return I.inv()
col_permutation(Matrix([[1, 4, 7], [2, 5, 8], [3, 6, 9]]))


Matrix([
[0, 0, 1],
[1, 0, 0],
[0, 1, 0]])

In [29]:

def random_del_columns(k,l,matrix):
    col_indices = random.sample(range(matrix.cols), k-l)#ramdonly rm k-l cols
    for i in col_indices:
        matrix.col_del(i)

    return matrix

random_del_columns(3,2,Matrix([[1, 2, 3], [4, 5, 6], [7, 8, 9]]))

Matrix([
[1, 3],
[4, 6],
[7, 9]])

In [103]:
def multiply_out(sample_M):
    for i in range(sample_M.rows):
        for j in range(sample_M.cols):
            sample_M[i,j]=expand(sample_M[i,j])
    return sample_M

multiply_out(Matrix([[1, 4, 7], [2, 5, 8], [3, 6, 9]]))

Matrix([
[1, 4, 7],
[2, 5, 8],
[3, 6, 9]])

In [33]:
def matrix_kl(k,l,s,n,t,B,q):#k,l are the dimensions of the matrix, s is the several, a small number
    Ls=[]
    Us=[]
    for i in range(s):
        U=generate_U(k,n,t,B,q)
        L=generate_L(k,n,t,B,q)
        Us.append(U)
        Ls.append(L)
        
    U_new=random.choice(Us)
    L_new=random.choice(Ls)
    Ms=Us+Ls
    
    P1=col_permutation(U_new)
    T1=random.choice(Ms)
    P2=col_permutation(U_new)
    T2=random.choice(Ms)
    P3=col_permutation(U_new)
    
    kk_matrix=U_new*L_new*P1*T1*P2*T2*P3
    S_inv=P3.inv(method="LU")*T2.inv(method="LU")*P2.inv(method="LU")*T1.inv(method="LU")*P1.inv(method="LU")*L_new.inv(method="LU")*U_new.inv(method="LU")
    
    kl_matrix=random_del_columns(k,l,kk_matrix)
    return S_inv,kl_matrix


A,B=matrix_kl(4,3,4,32,3,4,6)
B.shape


(4, 3)

# Finding the inverse

In [28]:
M2=Matrix([[2,1],[7,4]])
M2.inv(method="LU")

M2.adjugate()#the transpose of the matrix of cofactors.
det(M2)

1

In [20]:
def vector_t_polynomials(n,l,t,b,q): #l is the number of polynomials, q the remainder of 256%l
    U=[]
    for i in range(l):
        P=sparse_polynomial(n,t,b,q+1)#coefficient of original sparse_polynomials goes from 1 to q-1
        U.append(P)
    return U

vector_t_polynomials(32,10,5,5,6)

[5*x0*x10*x16*x18*x4 + 4*x11*x13*x15*x20 + 5*x14 + x18*x26*x5 + 6*x24*x9,
 4*x0*x12*x26*x27*x29 + 3*x0*x14*x27**2*x29 + 2*x1*x23*x25*x3*x7 + 6*x10**3*x6*x8 + 6*x12*x25*x28*x3,
 4*x0 + 3*x12*x14 + 3*x18 + 4*x19*x27 + 6*x4,
 3*x12*x17*x27*x6*x8 + 5*x15*x20 + 3*x21*x9 + 6*x30*x4 + 6*x4*x6,
 2*x10*x2*x3 + 8*x14 + 2*x24*x5*x8 + 5*x25,
 6*x11*x16*x2 + 6*x12*x16*x22*x3 + 6*x14**2*x17*x7*x9 + 4*x19*x20*x21*x24*x7 + x25*x7,
 x10 + x25*x29 + 2*x3 + 4*x30*x7 + 4*x9,
 6*x0*x19*x20*x23*x6 + 2*x1*x2*x23*x3 + 4*x16*x24*x30*x6 + 3*x2*x21 + 5*x2*x25*x6,
 3*x11*x12*x22*x27 + 4*x11*x13*x7 + 6*x15*x18*x24*x25*x31 + x22 + 5*x30,
 2*x10*x14*x15 + 4*x10*x19 + x12*x18*x22*x30*x5 + 6*x12*x8 + 6*x13*x15*x21*x4*x6]